# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
import pandas as pd
import warnings
from sqlalchemy import create_engine
nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'averaged_perceptron_tagger'])
from nltk import sent_tokenize, pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [44]:
# load data from database
engine = create_engine('sqlite:///C:/Users/giris/Documents/Analytics and Modelling/Disaster_Response_Pipeline/data/DisasterResponse.db')
df = pd.read_sql_table("Disaster_data",engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [45]:
def tokenize(text):
    """
    Tokenize the text by replacing URLs, removing non-alphanumeric characters, and converting to lowercase.
    
    Args:
    text: The input text to be tokenized.
    
    Returns:
    list: A list of clean tokens.
    """
    
    # define URL regex pattern
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Replace URLs with a placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # Remove all non-alphanumeric characters and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize and clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [46]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Custom transformer that extracts the starting verb of a sentence if there is one

    """

    def starting_verb(self, text):
        """
        Determine if the first word of any sentence in the input text is a verb.
        
        Args:
        text: The input text to be analyzed.
        
        Returns:
        bool: True if any sentence in the input text starts with a verb, otherwise False.
        """
        
        # Tokenize the text into sentences
        sentence_list = sent_tokenize(text)
        
        # Iterate through each sentence
        for sentence in sentence_list:
            # Tokenize and POS-tag the sentence
            pos_tags = pos_tag(tokenize(sentence))
            
            # Check if the list is empty and continue with the next sentence if so
            if not pos_tags:
                continue
            
            # Get the first word and its POS-tag
            first_word, first_tag = pos_tags[0]
            
            # Check if the first word is a verb or 'RT' (retweet)
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        
        # If no sentence starts with a verb, return False
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """
        Apply the starting_verb method to each element in the input data
        
        Args:
        X: input data to be transformed
        
        Returns:
        DataFrame: dataFrame containing the results of applying starting_verb to each element in X
        """
        
        # Apply the starting_verb method to each element in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        
        # Convert the series to a DataFrame and return
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
pipeline = Pipeline([
    # FeatureUnion combines the results of multiple transformer objects
    ('feature', FeatureUnion([
        # Pipeline for processing the text
        ('text', Pipeline([
            # Convert the text data into a matrix of token counts using a custom tokenizer function
            ('vect', CountVectorizer(tokenizer=tokenize)),
            # Apply Term Frequency-Inverse Document Frequency (TF-IDF) normalization
            ('tfidf', TfidfTransformer())
        ])),
        # Custom transformer for extracting the starting verb of a sentence, if there is one
        ('starting_verb', StartingVerbExtractor())
    ])),
    # Use a MultiOutputClassifier with a RandomForestClassifier as the base estimator for handling multi-label classification tasks
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
# Split the dataset into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [27]:
# fit te pipeline on the training data
pipeline.fit(X_train, y_train)
# predictions on test data
y_pred = pipeline.predict(X_test) 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
# prints classification report which includes evaluation metrics
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values ))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5050
               request       0.89      0.43      0.58      1079
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.63      0.69      2706
          medical_help       0.76      0.06      0.10       526
      medical_products       0.75      0.05      0.10       340
     search_and_rescue       0.64      0.04      0.07       197
              security       0.00      0.00      0.00       121
              military       0.92      0.05      0.10       217
                 water       0.87      0.28      0.42       412
                  food       0.87      0.50      0.64       726
               shelter       0.87      0.27      0.41       583
              clothing       0.82      0.09      0.17        95
                 money       1.00      0.03      0.05       148
        missing_people       1.00      

c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}
cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=2, n_jobs=-1)

# fit the pipeline on training data
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature',
                                        FeatureUnion(transformer_list=[('text',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000015DD98C8550>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
# predictions on test data
y_pred1 = cv.predict(X_test)

In [31]:
# prints classification report with the evaluation metrics
print(classification_report(y_test.values, y_pred1, target_names=Y.columns.values ))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5050
               request       0.89      0.44      0.59      1079
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.63      0.70      2706
          medical_help       0.64      0.04      0.08       526
      medical_products       0.80      0.05      0.09       340
     search_and_rescue       0.67      0.02      0.04       197
              security       0.00      0.00      0.00       121
              military       0.69      0.04      0.08       217
                 water       0.91      0.26      0.40       412
                  food       0.88      0.41      0.56       726
               shelter       0.87      0.30      0.44       583
              clothing       0.67      0.06      0.12        95
                 money       1.00      0.03      0.05       148
        missing_people       0.00      

c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
pipeline1 = Pipeline([
    ('feature', FeatureUnion([
    
        ('text', Pipeline([
            ('vect', HashingVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
    
    ])),

     ('starting_verb', StartingVerbExtractor())

    ])),
    
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


In [49]:
# fit the pipeline on training data
pipeline.fit(X_train, y_train)
# predictions on test data
y_pred2 = pipeline.predict(X_test) 

In [36]:
# print classification with the evalution metrics
print(classification_report(y_test.values, y_pred2, target_names=Y.columns.values ))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5050
               request       0.90      0.43      0.59      1079
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.63      0.70      2706
          medical_help       0.79      0.05      0.09       526
      medical_products       0.83      0.04      0.08       340
     search_and_rescue       0.56      0.03      0.05       197
              security       0.00      0.00      0.00       121
              military       0.79      0.05      0.10       217
                 water       0.93      0.26      0.40       412
                  food       0.88      0.45      0.59       726
               shelter       0.86      0.22      0.35       583
              clothing       0.60      0.06      0.11        95
                 money       1.00      0.03      0.05       148
        missing_people       0.00      

c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\giris\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.